### Prophet

In [19]:
import pandas as pd

In [20]:
from prophet import Prophet

In [21]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [37]:
train_data = pd.read_csv('snp_phophet_train.csv')
test_data = pd.read_csv('snp_phophet_test.csv')

In [38]:
train_data

,ds,stock,y
0,2018-01-02,A,0.000608
1,2018-01-03,A,0.025444
2,2018-01-04,A,-0.007501
3,2018-01-05,A,0.015988
4,2018-01-08,A,0.002146
...,...,...,...
547259,2022-04-21,ZTS,-0.016296
547260,2022-04-22,ZTS,-0.014996
547261,2022-04-25,ZTS,-0.009068
547262,2022-04-26,ZTS,-0.018469


In [39]:
prophet_models = {}

In [40]:
for stock in train_data['stock'].unique():
    # Filter data for the specific stock
    df_stock = train_data[train_data['stock'] == stock]

    # Create a new Prophet model for the stock
    model = Prophet()

    # Fit the model to the stock's data
    model.fit(df_stock)

    # Store the fitted model in the dictionary
    prophet_models[stock] = model

19:11:57 - cmdstanpy - INFO - Chain [1] start processing
19:11:57 - cmdstanpy - INFO - Chain [1] done processing
19:11:57 - cmdstanpy - INFO - Chain [1] start processing
19:11:57 - cmdstanpy - INFO - Chain [1] done processing
19:11:58 - cmdstanpy - INFO - Chain [1] start processing
19:11:58 - cmdstanpy - INFO - Chain [1] done processing
19:11:58 - cmdstanpy - INFO - Chain [1] start processing
19:11:58 - cmdstanpy - INFO - Chain [1] done processing
19:11:58 - cmdstanpy - INFO - Chain [1] start processing
19:11:58 - cmdstanpy - INFO - Chain [1] done processing
19:11:58 - cmdstanpy - INFO - Chain [1] start processing
19:11:58 - cmdstanpy - INFO - Chain [1] done processing
19:11:59 - cmdstanpy - INFO - Chain [1] start processing
19:11:59 - cmdstanpy - INFO - Chain [1] done processing
19:11:59 - cmdstanpy - INFO - Chain [1] start processing
19:11:59 - cmdstanpy - INFO - Chain [1] done processing
19:11:59 - cmdstanpy - INFO - Chain [1] start processing
19:11:59 - cmdstanpy - INFO - Chain [1]

predictions

In [41]:
stock_list =[]
for x in test_data['stock'].unique():
    stock_list.append(x)

In [42]:
# Initialize a DataFrame to store the Prophet predictions
prophet_pred = pd.DataFrame(columns=['Stock', 'MAE', 'RMSE'])

In [45]:
prophet_pred = pd.DataFrame(columns=['Stock', 'MAE', 'RMSE'])

for stock in stock_list:
    df_stock_test = test_data[test_data['stock'] == stock]
    model = prophet_models[stock]
    future = model.make_future_dataframe(periods=len(df_stock_test))
    forecast = model.predict(future)
    predicted_values = forecast[['ds', 'yhat']].tail(len(df_stock_test))
    actual_values = test_data[['ds', 'y']]
    actual_values['ds'] = pd.to_datetime(actual_values['ds'])
   
    evaluation_df = pd.merge(actual_values, predicted_values, on='ds', suffixes=('_actual', '_predicted'))
    
    # if evaluation_df.empty:
    #     continue  # Skip calculation if evaluation_df is empty
    
    mae = mean_absolute_error(evaluation_df['y'], evaluation_df['yhat'])
    rmse = mean_squared_error(evaluation_df['y'], evaluation_df['yhat'], squared=False)
    
    prophet_pred = prophet_pred.append({'Stock': stock, 'MAE': mae, 'RMSE': rmse}, ignore_index=True)
    


/tmp/ipykernel_5836/2216906520.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_values['ds'] = pd.to_datetime(actual_values['ds'])
/tmp/ipykernel_5836/2216906520.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  prophet_pred = prophet_pred.append({'Stock': stock, 'MAE': mae, 'RMSE': rmse}, ignore_index=True)
/tmp/ipykernel_5836/2216906520.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_values['

In [46]:
prophet_pred

,Stock,MAE,RMSE
0,A,0.016931,0.023581
1,AAL,0.017655,0.024358
2,AAP,0.017036,0.023687
3,AAPL,0.017210,0.023810
4,ABBV,0.017165,0.023788
...,...,...,...
498,YUM,0.016957,0.023524
499,ZBH,0.017012,0.023578
500,ZBRA,0.016820,0.023477
501,ZION,0.016957,0.023567


In [ ]:
prophet_pred.to_csv('prophet_pred.csv',index=False)